In [55]:
import math

import numpy as np
import pandas as pd

import lightgbm as lgb
import catboost as cat
from catboost import Pool
import xgboost as xgb

import itertools
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, BaseShuffleSplit, _validate_shuffle_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import log_loss

from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target

from scipy import stats

import eli5
from IPython.display import display
from eli5.permutation_importance import get_score_importances
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import seaborn as sns

import optuna

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


from colorama import Style, Fore

palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL


class CFG:
    undersample = True
    
    feature_sel = False
    n_feature_sel_folds = 5
    
    lgbm_optimize = True
    cb_optimize = True
    xgb_optimize = True
    n_trials = 1000
    n_optimize_folds = 3
    n_optimize_repeats = 5
    
    stacking = False
    n_stacking_folds = 10
    

# Load Data

In [2]:
# COMP_PATH = "/kaggle/input/icr-identify-age-related-conditions"
COMP_PATH = "icr-identify-age-related-conditions"

train_df = pd.read_csv(f'{COMP_PATH}//train.csv')
test_df = pd.read_csv(f'{COMP_PATH}/test.csv')
greeks = pd.read_csv(f"{COMP_PATH}/greeks.csv")
sample_submission = pd.read_csv(f"{COMP_PATH}/sample_submission.csv")

train_df['EJ'] = train_df['EJ'].replace({'A': 0, 'B': 1})
test_df['EJ'] = test_df['EJ'].replace({'A': 0, 'B': 1})

train_df.columns = train_df.columns.str.replace(' ', '')
test_df.columns = test_df.columns.str.replace(' ', '')

# train_df.drop('Id',axis=1, inplace=True)
# train_df.fillna(train_df.median(), inplace=True)

# Standard Scaler

In [3]:
# scaler = StandardScaler()
# new_num_cols = train_df.select_dtypes(include=['float64']).columns

# train_df[new_num_cols] = scaler.fit_transform(train_df[new_num_cols])
# test_df[new_num_cols] = scaler.transform(test_df[new_num_cols])

# Brute Force Feature Generation

Combine features in all possible ways.

In [4]:
# fi = pd.read_csv('feature_importances.csv', index_col = 'Unnamed: 0')
# fi_cols = set(fi['Feature'].head(100).values)

# perm = pd.read_csv('perm_df.csv', index_col = 'Unnamed: 0')
# perm_cols = set(perm['importance'].head(100).index)

# important_col = list(perm_cols.intersection(fi_cols))
# print(important_col)

# Denoising

In [5]:
# features = [fe for fe in train_df.columns if fe not in ['Id','CF', 'CB', 'DV', 'BR', 'DF', 'AR', 'GI', 'AY', 'GB',
#                                                         'AH', 'CW', 'CL', 'Class', 'BP']]

# for f in features:
#     train_df[f] = np.floor(train_df[f]*1000)/1000 # quality decreases no significant result for LGBM

# Log features (preserve sign)

In [6]:
# for f in features:
#     train_df[f] = np.sign(train_df[f]) * np.log1p(np.abs(train_df[f])) # no significant result for LGBM

# Remove outliers

In [7]:
features_with_outliers = [fe for fe in train_df.columns if fe not in ['BN', 'BQ', 'CW', 'EL', 'GH', 
                                                                      'GI', 'GL', 'Id', 'Class', 'EJ']]

for f in features_with_outliers:
    train_df[f] = train_df[f].clip(upper=train_df[f].quantile(0.99))

# Feature generation

In [8]:
features = train_df.drop(['Class', 'Id'], axis=1).columns

# features = [fe for fe in train_df.columns if fe not in ['CF', 'CB', 'DV', 'BR', 'DF', 'GB', 'AH', 
#                                                         'CW', 'CL', 'BP', 'BD', 'FC', 'GE', 'GF',
#                                                         'AR', 'GI', 'Id', 'Class', 'AX']]

# def gen_features(features, df):
#     generated_features = pd.DataFrame()

#     for fe_a, fe_b in tqdm(itertools.combinations(features, 2), total=sum([1 for i in itertools.combinations(features, 2)])):

# #         generated_features[f'{fe_a}_2']        = df[fe_a].pow(2)
# #         generated_features[f'{fe_b}_2']        = df[fe_b].pow(2)
# #         generated_features[f'{fe_a}*{fe_b}_2'] = df[fe_a] * df[fe_b].pow(2)
# #         generated_features[f'{fe_a}_2*{fe_b}'] = df[fe_a].pow(2) * df[fe_b]

# #         generated_features[f'{fe_a}_05'] = df[fe_a].pow(0.5)
# #         generated_features[f'{fe_b}_05'] = df[fe_b].pow(0.5)
# #         generated_features[f'{fe_a}*{fe_b}_05'] = df[fe_a] * df[fe_b].pow(0.5)
# #         generated_features[f'{fe_a}_05*{fe_b}'] = df[fe_a].pow(0.5) * df[fe_b]

# #         generated_features[f'{fe_a}_log'] = np.log(df[fe_a])
# #         generated_features[f'{fe_b}_log'] = np.log(df[fe_b])
# #         generated_features[f'{fe_a}*{fe_b}_log'] = df[fe_a] * np.log(df[fe_b])
# #         generated_features[f'{fe_a}_log*{fe_b}'] = np.log(df[fe_a]) * df[fe_b]
        
#     generated_features = generated_features[selected]
#     generated_features = pd.concat([generated_features, df[features]], axis=1)
    
#     # prevent inf
#     for g in generated_features.columns:
#         generated_features[g] = np.minimum(np.maximum(generated_features[g], -1e9), 1e9)
    
#     return generated_features

# generated_features_train = gen_features(features, train_df)
# generated_features_test = gen_features(features, test_df)

# features = generated_features_train.columns

# Add distance features

In [9]:
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier

class_imbalance = train_df[train_df['Class'] == 0].shape[0] / train_df[train_df['Class'] == 1].shape[0]

# average label of 20 Nearest Neighbours (colsine distance)
knn = NearestNeighbors(n_neighbors=21, metric='cosine', n_jobs=-1)
knn.fit(train_df[features].fillna(0))

# train
dists, nears = knn.kneighbors(train_df[features].fillna(0), return_distance=True)
dists, nears = dists[:,1:], nears[:,1:]

classes = np.array([train_df.loc[n, 'Class'] for n in nears])
train_df['class_cos'] = np.array(classes[i].mean() * class_imbalance for i in range(len(nears)))
train_df['class_cos'] = train_df['class_cos'].astype(float)

# test
dists, nears = knn.kneighbors(test_df[features].fillna(0), return_distance=True)
dists, nears = dists[:,1:], nears[:,1:]

classes = np.array([train_df.loc[n, 'Class'] for n in nears])
test_df['class_cos'] = np.array(classes[i].mean()  * class_imbalance for i in range(len(nears)))
test_df['class_cos'] = test_df['class_cos'].astype(float)


# features = [fe for fe in train_df.columns if fe not in ['CF', 'CB', 'DV', 'BR', 'DF', 'GB', 'AH', 
#                                                         'CW', 'CL', 'BP', 'BD', 'FC', 'GE', 'GF',
#                                                         'AR', 'GI', 'Id', 'Class', 'AX']]

# Multilabel Stratification

In [10]:
def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

        for sample_idx in sample_idxs:
            # Find the subset(s) with the largest number of desired examples
            # for this label, breaking ties by considering the largest number
            # of desired examples, breaking further ties randomly
            label_folds = c_folds_labels[:, label_idx]
            fold_idx = np.where(label_folds == label_folds.max())[0]

            if fold_idx.shape[0] > 1:
                temp_fold_idx = np.where(c_folds[fold_idx] ==
                                         c_folds[fold_idx].max())[0]
                fold_idx = fold_idx[temp_fold_idx]

                if temp_fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

            test_folds[sample_idx] = fold_idx
            labels_not_processed_mask[sample_idx] = False

            # Update desired number of examples
            c_folds_labels[fold_idx, labels[sample_idx]] -= 1
            c_folds[fold_idx] -= 1

    return test_folds


class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, *, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)


class RepeatedMultilabelStratifiedKFold(_RepeatedSplits):
    """Repeated Multilabel Stratified K-Fold cross validator.
    Repeats Mulilabel Stratified K-Fold n times with different randomization
    in each repetition.
    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.
    n_repeats : int, default=10
        Number of times cross-validator needs to be repeated.
    random_state : None, int or RandomState, default=None
        Random state to be used to generate random state for each
        repetition as well as randomly breaking ties within the iterative
        stratification algorithm.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2, n_repeats=2,
    ...     random_state=0)
    >>> for train_index, test_index in rmskf.split(X, y):
    ...     print("TRAIN:", train_index, "TEST:", test_index)
    ...     X_train, X_test = X[train_index], X[test_index]
    ...     y_train, y_test = y[train_index], y[test_index]
    ...
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [0 1 4 5] TEST: [2 3 6 7]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    See also
    --------
    RepeatedStratifiedKFold: Repeats (Non-multilabel) Stratified K-Fold
    n times.
    """
    def __init__(self, n_splits=5, *, n_repeats=10, random_state=None):
        super(RepeatedMultilabelStratifiedKFold, self).__init__(
            MultilabelStratifiedKFold, n_repeats=n_repeats, random_state=random_state,
            n_splits=n_splits)


class MultilabelStratifiedShuffleSplit(BaseShuffleSplit):
    """Multilabel Stratified ShuffleSplit cross-validator
    Provides train/test indices to split data into train/test sets.
    This cross-validation object is a merge of MultilabelStratifiedKFold and
    ShuffleSplit, which returns stratified randomized folds for multilabel
    data. The folds are made by preserving the percentage of each label.
    Note: like the ShuffleSplit strategy, multilabel stratified random splits
    do not guarantee that all folds will be different, although this is
    still very likely for sizeable datasets.
    Parameters
    ----------
    n_splits : int, default 10
        Number of re-shuffling & splitting iterations.
    test_size : float, int, None, optional
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, the value is set to 0.1.
        The default will change in version 0.21. It will remain 0.1 only
        if ``train_size`` is unspecified, otherwise it will complement
        the specified ``train_size``.
    train_size : float, int, or None, default is None
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split. If
        int, represents the absolute number of train samples. If None,
        the value is automatically set to the complement of the test size.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedShuffleSplit that only uses
        random_state when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.5,
    ...    random_state=0)
    >>> msss.get_n_splits(X, y)
    3
    >>> print(mss)       # doctest: +ELLIPSIS
    MultilabelStratifiedShuffleSplit(n_splits=3, random_state=0, test_size=0.5,
                                     train_size=None)
    >>> for train_index, test_index in msss.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    TRAIN: [1 2 5 6] TEST: [0 3 4 7]
    Notes
    -----
    Train and test sizes may be slightly different from desired due to the
    preference of stratification over perfectly sized folds.
    """

    def __init__(self, n_splits=10, *, test_size="default", train_size=None,
                 random_state=None):
        super(MultilabelStratifiedShuffleSplit, self).__init__(
            n_splits=n_splits, test_size=test_size, train_size=train_size, random_state=random_state)

    def _iter_indices(self, X, y, groups=None):
        n_samples = _num_samples(X)
        y = check_array(y, ensure_2d=False, dtype=None)
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(
                    type_of_target_y))

        n_train, n_test = _validate_shuffle_split(n_samples, self.test_size,
                                                  self.train_size)

        n_samples = y.shape[0]
        rng = check_random_state(self.random_state)
        y_orig = y.copy()

        r = np.array([n_train, n_test]) / (n_train + n_test)

        for _ in range(self.n_splits):
            indices = np.arange(n_samples)
            rng.shuffle(indices)
            y = y_orig[indices]

            test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

            test_idx = test_folds[np.argsort(indices)] == 1
            test = np.where(test_idx)[0]
            train = np.where(~test_idx)[0]

            yield train, test

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedShuffleSplit, self).split(X, y, groups)

# LGBM feature selection

In [45]:
from shaphypetune import BoostBoruta

params = {
        'boosting_type':'goss',
        'learning_rate': 0.06733232950390658, 
        'n_estimators': 5000, 
        'early_stopping_round' : 100, 
        'subsample' : 0.7, # bagging_fraction
        'colsample_bytree': 0.6, # feature_fraction
        'num_leaves': 33,
        'class_weight': 'balanced',
        'metric': 'none', 
        'is_unbalance': True, 
        'random_state': 8062023,
        'feature_fraction_seed': 8062023,
        'bagging_seed': 8062023,
        'max_depth': 6,
        'reg_alpha': 2.025436e-04,  
        'reg_lambda': 2.290193e-07,
#         'bagging_freq': 6,
        'max_bin': 198,
        'min_child_samples': 32,
        'importance_type': 'gain'
        }

def balanced_log_loss(y_true, y_pred):

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1 - y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def calc_log_loss_weight(y_true): 
    '''w0, w1 assign different weights to individual data points during training.'''
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

def lgbm_tuning(features, permut=False, boruta=False):
    metric = balanced_log_loss
    eval_results_ = {}

    cv_scores = [] # store all cv scores of outer loop inference

    perm_df_ = pd.DataFrame()
    feature_importances_ = pd.DataFrame()
    boruta_df_ = pd.DataFrame()
    
    for i in range(CFG.n_optimize_repeats):
        print(f'Repeat {blu}#{i+1}')
        
        # Make random under-sampling to balance classes
        positive_count_train = train_df.Class.value_counts()[1]
        sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                        1: positive_count_train}, 
                                     random_state=15062023+i, 
                                     replacement=True)

        X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']
        
        if CFG.undersample:
            X_re, y_re = sampler.fit_resample(X_re, y_re)
        
        # Create Stratified Multilabel k-Fold scheme
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_feature_sel_folds, shuffle=True, random_state=8062023+i)

        # Create an oof array for inner loop
        oof = np.zeros(X_re.shape[0])
        
        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X=X_re[features], y=X_re.iloc[:,-3:]), start = 1): 
            X, y = X_re[features], y_re

            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]


            X_train = X_train.reset_index(drop=True)
            y_train = y_train.reset_index(drop=True)

            # Store models here
            models_ = [] 

            eval_results_[fold]= {}

            clf = lgb.LGBMClassifier(**params)
            clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                    eval_metric=bll_metric, # eval_sample_weight=w_val, 
                    early_stopping_rounds=100, verbose=1)

            models_.append(clf)

            val_preds = clf.predict_proba(X_val)[:,1]
            oof[val_idx] = val_preds

            val_score = metric(y_val, val_preds)
            best_iter = clf.best_iteration_

            print(f'Fold: {blu}{fold:>3}{res}| {metric.__name__}: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')

            # permutation importance
            if permut:
                perm = PermutationImportance(clf, scoring=None, n_iter=1, 
                                             random_state=42, cv=None, refit=False).fit(X_val, y_val)

                perm_importance_df = pd.DataFrame({'importance': perm.feature_importances_}, 
                                                   index=X_val.columns).sort_index()

                if perm_df_.shape[0] == 0:
                    perm_df_ = perm_importance_df.copy()
                else:
                    perm_df_ += perm_importance_df

            # tree feature importance
            f_i = pd.DataFrame(sorted(zip(clf.feature_importances_, X.columns), 
                                              reverse=True, key=lambda x: x[1]), 
                               columns=['Value','Feature'])

            if feature_importances_.shape[0] == 0:
                feature_importances_ = f_i.copy()
            else:

                feature_importances_['Value'] += f_i['Value']

            # Boruta SHAP importance
            if boruta:
                model = BoostBoruta(clf, importance_type='shap_importances', train_importance=False)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                          eval_metric=bll_metric, early_stopping_rounds=300, verbose=-1)

                boruta_importance_df = pd.DataFrame({'importance': model.ranking_}, 
                                                     index=X_train.columns).sort_index()
                if boruta_df_.shape[0] == 0:
                    boruta_df_ = boruta_importance_df.copy()
                else:
                    boruta_df_ += boruta_importance_df

        fold_cv_score = metric(y_re, oof)
        print(f'{red} CV score: {res} {metric.__name__}: {red}{fold_cv_score:.5f}{res}')
        print(f'{"*" * 50}\n')
        cv_scores.append(fold_cv_score)


    print(f'{red} Avg score {CFG.n_feature_sel_folds}-fold: {res} {metric.__name__}: {red}{np.mean(cv_scores):.5f}{res}')
    print(f'{"*" * 50}\n')
    
    if permut:
        perm_df_ = perm_df_.sort_values('importance', ascending=False)
        
    if boruta:
        boruta_df_ = boruta_df_.sort_values('importance')
                                    
    feature_importances_ = feature_importances_.sort_values('Value', ascending=False)
    
    return perm_df_, feature_importances_, boruta_df_, np.mean(cv_scores)

if CFG.feature_sel:
    perm_df_, feature_importances_, boruta_df_, cv_scores = lgbm_tuning(features, permut=False, boruta=False)

Repeat #1
[1]	valid_0's balanced_log_loss: 0.675494
[2]	valid_0's balanced_log_loss: 0.663652
[3]	valid_0's balanced_log_loss: 0.652625
[4]	valid_0's balanced_log_loss: 0.623198
[5]	valid_0's balanced_log_loss: 0.607029
[6]	valid_0's balanced_log_loss: 0.581928
[7]	valid_0's balanced_log_loss: 0.564768
[8]	valid_0's balanced_log_loss: 0.546604
[9]	valid_0's balanced_log_loss: 0.537501
[10]	valid_0's balanced_log_loss: 0.518296
[11]	valid_0's balanced_log_loss: 0.505712
[12]	valid_0's balanced_log_loss: 0.503956
[13]	valid_0's balanced_log_loss: 0.492748
[14]	valid_0's balanced_log_loss: 0.483312
[15]	valid_0's balanced_log_loss: 0.483312
[16]	valid_0's balanced_log_loss: 0.483312
[17]	valid_0's balanced_log_loss: 0.483312
[18]	valid_0's balanced_log_loss: 0.483312
[19]	valid_0's balanced_log_loss: 0.483312
[20]	valid_0's balanced_log_loss: 0.483312
[21]	valid_0's balanced_log_loss: 0.483312
[22]	valid_0's balanced_log_loss: 0.483312
[23]	valid_0's balanced_log_loss: 0.483312
[24]	valid

[41]	valid_0's balanced_log_loss: 0.476896
[42]	valid_0's balanced_log_loss: 0.476896
[43]	valid_0's balanced_log_loss: 0.476896
[44]	valid_0's balanced_log_loss: 0.476896
[45]	valid_0's balanced_log_loss: 0.476896
[46]	valid_0's balanced_log_loss: 0.476896
[47]	valid_0's balanced_log_loss: 0.476896
[48]	valid_0's balanced_log_loss: 0.476896
[49]	valid_0's balanced_log_loss: 0.476896
[50]	valid_0's balanced_log_loss: 0.476896
[51]	valid_0's balanced_log_loss: 0.476896
[52]	valid_0's balanced_log_loss: 0.476896
[53]	valid_0's balanced_log_loss: 0.476896
[54]	valid_0's balanced_log_loss: 0.476896
[55]	valid_0's balanced_log_loss: 0.476896
[56]	valid_0's balanced_log_loss: 0.476896
[57]	valid_0's balanced_log_loss: 0.476896
[58]	valid_0's balanced_log_loss: 0.476896
[59]	valid_0's balanced_log_loss: 0.476896
[60]	valid_0's balanced_log_loss: 0.476896
[61]	valid_0's balanced_log_loss: 0.476896
[62]	valid_0's balanced_log_loss: 0.476896
[63]	valid_0's balanced_log_loss: 0.476896
[64]	valid_

[1]	valid_0's balanced_log_loss: 0.661572
[2]	valid_0's balanced_log_loss: 0.633734
[3]	valid_0's balanced_log_loss: 0.622212
[4]	valid_0's balanced_log_loss: 0.599433
[5]	valid_0's balanced_log_loss: 0.590956
[6]	valid_0's balanced_log_loss: 0.554559
[7]	valid_0's balanced_log_loss: 0.528836
[8]	valid_0's balanced_log_loss: 0.506323
[9]	valid_0's balanced_log_loss: 0.492589
[10]	valid_0's balanced_log_loss: 0.478391
[11]	valid_0's balanced_log_loss: 0.465595
[12]	valid_0's balanced_log_loss: 0.447033
[13]	valid_0's balanced_log_loss: 0.436092
[14]	valid_0's balanced_log_loss: 0.414396
[15]	valid_0's balanced_log_loss: 0.414396
[16]	valid_0's balanced_log_loss: 0.414396
[17]	valid_0's balanced_log_loss: 0.414396
[18]	valid_0's balanced_log_loss: 0.414396
[19]	valid_0's balanced_log_loss: 0.414396
[20]	valid_0's balanced_log_loss: 0.414396
[21]	valid_0's balanced_log_loss: 0.414396
[22]	valid_0's balanced_log_loss: 0.414396
[23]	valid_0's balanced_log_loss: 0.414396
[24]	valid_0's balan

[10]	valid_0's balanced_log_loss: 0.524836
[11]	valid_0's balanced_log_loss: 0.507324
[12]	valid_0's balanced_log_loss: 0.495464
[13]	valid_0's balanced_log_loss: 0.483312
[14]	valid_0's balanced_log_loss: 0.47058
[15]	valid_0's balanced_log_loss: 0.47058
[16]	valid_0's balanced_log_loss: 0.47058
[17]	valid_0's balanced_log_loss: 0.47058
[18]	valid_0's balanced_log_loss: 0.47058
[19]	valid_0's balanced_log_loss: 0.47058
[20]	valid_0's balanced_log_loss: 0.47058
[21]	valid_0's balanced_log_loss: 0.47058
[22]	valid_0's balanced_log_loss: 0.47058
[23]	valid_0's balanced_log_loss: 0.47058
[24]	valid_0's balanced_log_loss: 0.47058
[25]	valid_0's balanced_log_loss: 0.47058
[26]	valid_0's balanced_log_loss: 0.47058
[27]	valid_0's balanced_log_loss: 0.47058
[28]	valid_0's balanced_log_loss: 0.47058
[29]	valid_0's balanced_log_loss: 0.47058
[30]	valid_0's balanced_log_loss: 0.47058
[31]	valid_0's balanced_log_loss: 0.47058
[32]	valid_0's balanced_log_loss: 0.47058
[33]	valid_0's balanced_log_lo

[1]	valid_0's balanced_log_loss: 0.660084
[2]	valid_0's balanced_log_loss: 0.632845
[3]	valid_0's balanced_log_loss: 0.614469
[4]	valid_0's balanced_log_loss: 0.591348
[5]	valid_0's balanced_log_loss: 0.570386
[6]	valid_0's balanced_log_loss: 0.551285
[7]	valid_0's balanced_log_loss: 0.536158
[8]	valid_0's balanced_log_loss: 0.52579
[9]	valid_0's balanced_log_loss: 0.516911
[10]	valid_0's balanced_log_loss: 0.50592
[11]	valid_0's balanced_log_loss: 0.494012
[12]	valid_0's balanced_log_loss: 0.48363
[13]	valid_0's balanced_log_loss: 0.476049
[14]	valid_0's balanced_log_loss: 0.469509
[15]	valid_0's balanced_log_loss: 0.469509
[16]	valid_0's balanced_log_loss: 0.469509
[17]	valid_0's balanced_log_loss: 0.469509
[18]	valid_0's balanced_log_loss: 0.469509
[19]	valid_0's balanced_log_loss: 0.469509
[20]	valid_0's balanced_log_loss: 0.469509
[21]	valid_0's balanced_log_loss: 0.469509
[22]	valid_0's balanced_log_loss: 0.469509
[23]	valid_0's balanced_log_loss: 0.469509
[24]	valid_0's balanced

[13]	valid_0's balanced_log_loss: 0.45206
[14]	valid_0's balanced_log_loss: 0.440944
[15]	valid_0's balanced_log_loss: 0.440944
[16]	valid_0's balanced_log_loss: 0.440944
[17]	valid_0's balanced_log_loss: 0.440944
[18]	valid_0's balanced_log_loss: 0.440944
[19]	valid_0's balanced_log_loss: 0.440944
[20]	valid_0's balanced_log_loss: 0.440944
[21]	valid_0's balanced_log_loss: 0.440944
[22]	valid_0's balanced_log_loss: 0.440944
[23]	valid_0's balanced_log_loss: 0.440944
[24]	valid_0's balanced_log_loss: 0.440944
[25]	valid_0's balanced_log_loss: 0.440944
[26]	valid_0's balanced_log_loss: 0.440944
[27]	valid_0's balanced_log_loss: 0.440944
[28]	valid_0's balanced_log_loss: 0.440944
[29]	valid_0's balanced_log_loss: 0.440944
[30]	valid_0's balanced_log_loss: 0.440944
[31]	valid_0's balanced_log_loss: 0.440944
[32]	valid_0's balanced_log_loss: 0.440944
[33]	valid_0's balanced_log_loss: 0.440944
[34]	valid_0's balanced_log_loss: 0.440944
[35]	valid_0's balanced_log_loss: 0.440944
[36]	valid_0

# Check features correlation

In [12]:
if CFG.feature_sel:
    col = 'BZ'
    x = train_df[train_df[col] <= train_df[col].quantile(0.99)]
    cm = x[[c for c in train_df.columns if c not in ['Id', 'Class']]].corr()
    display(np.abs(cm[col]).sort_values(ascending=False)[1:])

# Analyze permutation feature importance

In [13]:
if CFG.feature_sel:
    perm_df_.to_csv('perm_df.csv')
    perm_df_
    perm_cols = set(perm_df_.index[-35:])
    display(perm_cols)

# Analyze tree gain feature importance

In [14]:
if CFG.feature_sel:
    feature_importances_.to_csv('feature_importances.csv')
    feature_importances_
    fi_cols = set(feature_importances_['Feature'].values[-23:])
    display(fi_cols)

# Analyze BORUTA importance

In [15]:
if CFG.feature_sel:
    boruta_df_.to_csv('boruta_df_.csv')
    boruta_df_
    boruta_cols = set(boruta_df_.index[-35:])
    display(boruta_cols)

# LGBM Optuna optimization

In [54]:
def balanced_log_loss(y_true, y_pred):
    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1 - y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

def bll_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def calc_log_loss_weight(y_true): 
    '''w0, w1 assign different weights to individual data points during training.'''
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

X, y = train_df[features], train_df.Class 
    
def objective(trial):
    param = {
        # Main parameters
#                     'device': 'gpu',
#                     'gpu_platform_id': 0,
#                     'gpu_device_id': 0,
        'objective': 'binary',
        'metric': 'none',
        'is_unbalance': True,
        'boosting_type': trial.suggest_categorical('boosting_type', ['goss', 'gbdt', 'dart']),   
        # Hyperparamters (in order of importance decreasing)
        'n_estimators': 3000, # trial.suggest_int('n_estimators', 500, 1500),  # max number of trees in model
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 3e-1),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True), # L1,  alias: reg_alpha
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True), # L2, alias: reg_lambda
         # decrease to deal with overfit
        'max_depth': trial.suggest_int('max_depth', 4, 10),   # tree max depth 
         # decrease to deal with overfit
        'num_leaves': trial.suggest_int('num_leaves', 4, 128),  # Max number of leaves in one tree
                                                               # should be ~ 2**(max_depth-1)
        'bagging_fraction': None, # Randomly select part of data without 
                                  # resampling if bagging_fraction < 1.0
                                  # alias: subsample
        'feature_fraction': trial.suggest_float('feature_fraction', 0.3, 0.7), # Randomly select a subset of features 
                                                                   # if feature_fraction < 1.0
                                                                   # alias: colsample_bytree
        # decrease to deal with overfit
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100), # Minimal number of data in one leaf
                                                                           # aliases: min_child_samples, 
#             # decrease to deat with overfit
#             'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-4, 1e-1), # Stop trying to split 
#                                                                                                    # leave if sum of it's
#                                                                                                    # hessian less than k
#                                                                                                    # alias: min_child_weight

        # increase for accuracy, decrease to deal with overfit
        'max_bin': trial.suggest_int('max_bin', 32, 255), # Max number of bins that feature values will be bucketed in
        # increase to deal with overfit
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7), # Perform bagging at every k iteration
        'early_stopping_round': 100, 

#           'subsample_for_bin': 200000, # Number of data that sampled to construct feature discrete bins; setting this 
                                     # to larger value will give better training result but may increase train time 
#           'cat_smooth': trial.suggest_float('cat_smooth', 10.0, 100.0),  # this can reduce the effect of noises in 
                                                                       # categorical features, especially for 
                                                                       # categories with few data
        'verbose': -1
    }

    if param['boosting_type'] != 'goss':
        param['bagging_fraction'] = trial.suggest_float('bagging_fraction', 0.3, 0.7)

    bll_list = list()
    
    for i in range(CFG.n_optimize_repeats):
        print(f'Repeat {blu}#{i+1}')

        # Make random under-sampling to balance classes
        positive_count_train = train_df.Class.value_counts()[1]
        sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                        1: positive_count_train}, 
                                     random_state=15062023+i, 
                                     replacement=True)

        X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']
        
        if CFG.undersample:
            X_re, y_re = sampler.fit_resample(X_re, y_re)
        
        # Create Stratified Multilabel k-Fold scheme
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_feature_sel_folds, shuffle=True, random_state=10062023+i)

        # Create an oof array for inner loop
        oof = np.zeros(X_re.shape[0])

        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X=X_re[features], y=X_re.iloc[:,-3:]), start=1): 
            X, y = X_re[features], y_re
            
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            dtrain = lgb.Dataset(X_train, label=y_train)
            dvalid = lgb.Dataset(X_val, label=y_val)

            # Add a callback for pruning
#             pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'balanced_log_loss')

            gbm = lgb.train(
                param, dtrain, valid_sets=[dvalid], # callbacks=[pruning_callback], 
                feval=bll_metric, verbose_eval=0
            )

            val_preds = gbm.predict(X_val)
            oof[val_idx] = val_preds
        bll_list.append(balanced_log_loss(y_re, oof))

    return np.mean(bll_list)
            

if CFG.lgbm_optimize:
#     study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=100), direction="minimize")
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=CFG.n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe().sort_values('value')
    df.to_csv(f'optuna_lgbm_{boosting_type}.csv')

[I 2023-06-15 15:20:15,455] A new study created in memory with name: no-name-b4491000-b388-4f73-8d8a-7f27ff9f3a2a


Repeat #1
Repeat #2
Repeat #3
Repeat #4
Repeat #5


[I 2023-06-15 15:20:17,853] Trial 0 finished with value: 0.18892005719012475 and parameters: {'boosting_type': 'goss', 'learning_rate': 0.10697234520758851, 'lambda_l1': 0.000396964712338032, 'lambda_l2': 0.5452195131263328, 'max_depth': 10, 'num_leaves': 123, 'feature_fraction': 0.31233003634052664, 'min_data_in_leaf': 18, 'max_bin': 113, 'bagging_freq': 5}. Best is trial 0 with value: 0.18892005719012475.


Repeat #1
Repeat #2
Repeat #3
Repeat #4
Repeat #5


[I 2023-06-15 15:20:59,925] Trial 1 finished with value: 0.32141918764421884 and parameters: {'boosting_type': 'dart', 'learning_rate': 0.01291886257117534, 'lambda_l1': 7.971757418525103e-07, 'lambda_l2': 0.033246695034243774, 'max_depth': 10, 'num_leaves': 26, 'feature_fraction': 0.5919810217300694, 'min_data_in_leaf': 23, 'max_bin': 171, 'bagging_freq': 4, 'bagging_fraction': 0.326440880874238}. Best is trial 0 with value: 0.18892005719012475.


...

Repeat #5
Repeat #1
Repeat #2
Repeat #3
Repeat #4
Repeat #5


[I 2023-06-15 15:21:12,888] Trial 7 finished with value: 0.6973915667428459 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.06292288328578349, 'lambda_l1': 0.26821685232687464, 'lambda_l2': 9.730379881711538e-06, 'max_depth': 5, 'num_leaves': 112, 'feature_fraction': 0.5173379162150782, 'min_data_in_leaf': 78, 'max_bin': 123, 'bagging_freq': 6, 'bagging_fraction': 0.5824817764566992}. Best is trial 0 with value: 0.18892005719012475.


Repeat #1


[W 2023-06-15 15:21:14,316] Trial 8 failed with parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.0006303332783493984, 'lambda_l1': 2.6605054586252357, 'lambda_l2': 1.1945871999575372e-07, 'max_depth': 4, 'num_leaves': 48, 'feature_fraction': 0.32667953679014455, 'min_data_in_leaf': 18, 'max_bin': 238, 'bagging_freq': 6, 'bagging_fraction': 0.35721817408643103} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/alex/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_5861/4055452463.py", line 117, in objective
    gbm = lgb.train(
  File "/home/alex/.local/lib/python3.10/site-packages/lightgbm/engine.py", line 292, in train
    booster.update(fobj=fobj)
  File "/home/alex/.local/lib/python3.10/site-packages/lightgbm/basic.py", line 3021, in update
    _safe_call(_LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt
[W 2023-06-15 15:21:14,316

KeyboardInterrupt: 

# Load LGBM parameters

In [18]:
import glob

param_list = glob.glob("optuna_lgbm*.csv")
models = list()
best_lgbm_params = list()

lgbm_params = pd.DataFrame()

for f in param_list:
    gb_type = [f.split('_')][0][2][:-4]
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    tmp['params_boosting_type'] = gb_type
    if lgbm_params.shape[0] == 0:
        lgbm_params = tmp
    else:
        lgbm_params = pd.concat([lgbm_params, tmp])
        
lgbm_params = lgbm_params.sort_values('value').head(20)
param_cols = [c for c in lgbm_params.columns if c.startswith('params_')]
lgbm_params = lgbm_params[param_cols]

for idx, row in lgbm_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['objective'] = 'binary'
    row_dict['metric'] = 'none'
#     row_dict['subsample_for_bin'] = 300000
    row_dict['force_col_wise'] = False
    row_dict['early_stopping_rounds'] = 50
    row_dict['verbose'] = -1
    row_dict['max_bin'] = 255
    row_dict['bagging_freq'] = int(row_dict['bagging_freq'])
#     if row_dict['bagging_fraction'] != row_dict['bagging_fraction']:
#         row_dict['bagging_fraction'] = None
    row_dict['min_data_in_leaf'] = int(row_dict['min_data_in_leaf'])
    row_dict['n_estimators'] = 3000 # int(row_dict['n_estimators'])
    
    row_dict['learning_rate'] = 0.06433232950390658 # float(row_dict['learning_rate'])
    row_dict['num_leaves'] = int(row_dict['num_leaves'])
    row_dict['max_depth'] = int(row_dict['max_depth'])
    row_dict['is_unbalance'] = True
    row_dict['class_weight'] = 'balanced'
    row_dict['verbose'] = -1
    
    best_lgbm_params.append(row_dict)

# LGBM train

In [19]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def lgbm_training():
    models_ = list()
    bll_list = list()
    weights_ = list()
    
    X, y = train_df[features], train_df.Class
#     X, y = generated_features_train, train_df.Class
     
    kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=8062023+20)
    metric = balanced_log_loss
    eval_results_ = {}     # used to store evaluation results for each fold

    oof_level2 = np.zeros([y.shape[0], len(best_lgbm_params) + 1])
    oof_level2[:, len(best_lgbm_params)] = y

    print(f"Training with {blu}{X.shape[1]}{res} features")

    for fold, (fit_idx, val_idx) in tqdm(enumerate(kf.split(X=train_df, y=greeks.iloc[:,1:3]), start = 1),
                                         total=CFG.n_stacking_folds):
        
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]

        for i, params in enumerate(best_lgbm_params):
            
            clf = lgb.LGBMClassifier(**params)
            clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
                    eval_metric=bll_metric, verbose=-1)
            models_.append(clf)

            val_preds = clf.predict_proba(X_val)[:,1]
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = clf.best_iteration_

            print(clf.best_iteration_)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')
        
    return oof_level2, models_

if CFG.stacking:
    oof_level2_lgbm, models_lgbm = lgbm_training()

Training with 39 features


  0%|          | 0/10 [00:00<?, ?it/s]

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=1.7964370113205523e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.7964370113205523e-08
[LightGBM] [Warning] lambda_l2 is set=1.7522707651576903e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7522707651576903e-06
[LightGBM] [Warning] feature_fraction is set=0.6246598507060309, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6246598507060309
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
488
Fold:   1| bll_metric: 0.01764 | Best iteration:  488
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightG

599
Fold:   1| bll_metric: 0.01536 | Best iteration:  599
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=1.667715087832313e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.667715087832313e-06
[LightGBM] [Warning] lambda_l2 is set=2.4808900057624336e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4808900057624336e-07
[LightGBM] [Warning] feature_fraction is set=0.3700160061529473, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3700160061529473
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
600
Fold:   1| bll_metric: 0.01629 | Best iteration:  600
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be

392
Fold:   2| bll_metric: 0.12295 | Best iteration:  392
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=1.841201567781372e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.841201567781372e-08
[LightGBM] [Warning] lambda_l2 is set=3.538151412119281e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.538151412119281e-06
[LightGBM] [Warning] feature_fraction is set=0.6068378806689372, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6068378806689372
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
219
Fold:   2| bll_metric: 0.12824 | Best iteration:  219
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be i

...

379
Fold:   8| bll_metric: 0.04460 | Best iteration:  379
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=1.667715087832313e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.667715087832313e-06
[LightGBM] [Warning] lambda_l2 is set=2.4808900057624336e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4808900057624336e-07
[LightGBM] [Warning] feature_fraction is set=0.3700160061529473, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3700160061529473
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
733
Fold:   8| bll_metric: 0.01666 | Best iteration:  733
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be

186
Fold:   9| bll_metric: 0.13942 | Best iteration:  186
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=0.0075204418283168, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0075204418283168
[LightGBM] [Warning] lambda_l2 is set=8.664310638314929e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.664310638314929e-06
[LightGBM] [Warning] feature_fraction is set=0.6150903977071068, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6150903977071068
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
187
Fold:   9| bll_metric: 0.07333 | Best iteration:  187
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored

208
Fold:   9| bll_metric: 0.15718 | Best iteration:  208
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=0.0001946386713123, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0001946386713123
[LightGBM] [Warning] lambda_l2 is set=1.7477628916919897e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7477628916919897e-06
[LightGBM] [Warning] feature_fraction is set=0.5324680203813787, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5324680203813787
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
145
Fold:   9| bll_metric: 0.16605 | Best iteration:  145
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignor

115
Fold:  10| bll_metric: 0.38013 | Best iteration:  115
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=0.0001480696470031, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0001480696470031
[LightGBM] [Warning] lambda_l2 is set=9.27322014998632e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.27322014998632e-06
[LightGBM] [Warning] feature_fraction is set=0.5288974232551956, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5288974232551956
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
105
Fold:  10| bll_metric: 0.36513 | Best iteration:  105
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. 

95
Fold:  10| bll_metric: 0.39734 | Best iteration:   95
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] lambda_l1 is set=0.0529539750398388, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0529539750398388
[LightGBM] [Warning] lambda_l2 is set=8.763924644936586e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.763924644936586e-06
[LightGBM] [Warning] feature_fraction is set=0.4795187637817097, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4795187637817097
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
53
Fold:  10| bll_metric: 0.37825 | Best iteration:   53
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Cu

# XGBoost Optuna optimization

In [57]:
X, y = train_df[features], train_df.Class

def objective(trial):
    bll_list = list()

    params = {
        "n_estimators": 3000, # trial.suggest_int('n_estimators', 100, 1000, step=100),
        "random_state": 14062023,
        "early_stopping_rounds": 100,
        "verbosity": 0,
        "scale_pos_weight": 4.71,
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear"]),# "dart"]), 
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
    }

    if params["booster"] in ["gbtree", "dart"]:
        params["learning_rate"] = trial.suggest_float("learning_rate", 1e-3, 0.1, log=True) # alias eta
        # maximum depth of the tree, signifies complexity of the tree.
        params["max_depth"] = trial.suggest_int("max_depth", 3, 10)
        # minimum child weight, larger the term more conservative the tree.
        params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        # defines how selective algorithm is.
        params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    for i in range(CFG.n_optimize_repeats):
        print(f'Repeat {blu}#{i+1}')

        # Make random under-sampling to balance classes
        positive_count_train = train_df.Class.value_counts()[1]
        sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                        1: positive_count_train}, 
                                     random_state=15062023+i, 
                                     replacement=True)

        X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']
        
        if CFG.undersample:
            X_re, y_re = sampler.fit_resample(X_re, y_re)
        
        # Create Stratified Multilabel k-Fold scheme
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_feature_sel_folds, shuffle=True, random_state=10062023+i)

        # Create an oof array for inner loop
        oof = np.zeros(X_re.shape[0])

        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X=X_re[features], y=X_re.iloc[:,-3:]), start=1): 
            X, y = X_re[features], y_re
            
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            # Learning
            model = xgb.XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1000)
            # Predict
            val_preds = model.predict_proba(X_val)[:,1]
            oof[val_idx] = val_preds
        
        bll_list.append(balanced_log_loss(y_re, oof))    
    
    return np.mean(bll_list)

if CFG.xgb_optimize:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=CFG.n_trials, )

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe()
    df.sort_values('value').iloc[:, [1] + list(range(5, 14))]
    df.to_csv(f'optuna_xgb.csv')

[I 2023-06-15 15:27:44,387] A new study created in memory with name: no-name-5b706e8c-7411-4867-babf-b8c6cb53825e


Repeat #1
[0]	validation_0-logloss:0.68057
[409]	validation_0-logloss:0.28587
[0]	validation_0-logloss:0.68524
[335]	validation_0-logloss:0.22760
[0]	validation_0-logloss:0.66400
[548]	validation_0-logloss:0.40614
[0]	validation_0-logloss:0.66888
[232]	validation_0-logloss:0.43345
[0]	validation_0-logloss:0.68240
[230]	validation_0-logloss:0.42186
Repeat #2
[0]	validation_0-logloss:0.64133
[434]	validation_0-logloss:0.10903
[0]	validation_0-logloss:0.65529
[231]	validation_0-logloss:0.30404
[0]	validation_0-logloss:0.65559
[639]	validation_0-logloss:0.27892
[0]	validation_0-logloss:0.65559
[128]	validation_0-logloss:0.43786
[0]	validation_0-logloss:0.65433
[562]	validation_0-logloss:0.24229
Repeat #3
[0]	validation_0-logloss:0.67084
[311]	validation_0-logloss:0.29251
[0]	validation_0-logloss:0.68051
[374]	validation_0-logloss:0.24343
[0]	validation_0-logloss:0.66691
[539]	validation_0-logloss:0.14530
[0]	validation_0-logloss:0.68407
[704]	validation_0-logloss:0.29233
[0]	validation_0-l

[I 2023-06-15 15:27:49,624] Trial 0 finished with value: 0.286093718239291 and parameters: {'booster': 'gbtree', 'alpha': 0.25623728735464407, 'lambda': 0.00019336903261921336, 'subsample': 0.7112340405466451, 'colsample_bytree': 0.6375939340861976, 'learning_rate': 0.0807215616583345, 'max_depth': 9, 'min_child_weight': 9, 'gamma': 0.204081655787243, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.286093718239291.


Repeat #1
[0]	validation_0-logloss:0.74057
[107]	validation_0-logloss:0.69313
[0]	validation_0-logloss:0.72642
[112]	validation_0-logloss:0.42529
[0]	validation_0-logloss:1.31727
[100]	validation_0-logloss:0.84068
[0]	validation_0-logloss:0.74713
[114]	validation_0-logloss:0.54952
[0]	validation_0-logloss:0.78410
[1000]	validation_0-logloss:0.46838
[1068]	validation_0-logloss:0.46838
Repeat #2
[0]	validation_0-logloss:0.74303
[1000]	validation_0-logloss:0.26996
[2000]	validation_0-logloss:0.26966
[2999]	validation_0-logloss:0.26965
[0]	validation_0-logloss:2.21567
[100]	validation_0-logloss:15.42197
[0]	validation_0-logloss:1.13108
[111]	validation_0-logloss:0.61508
[0]	validation_0-logloss:0.79442
[103]	validation_0-logloss:1.66532
[0]	validation_0-logloss:0.73432
[199]	validation_0-logloss:0.37096
Repeat #3
[0]	validation_0-logloss:0.70926
[111]	validation_0-logloss:0.75246
[0]	validation_0-logloss:1.99752
[100]	validation_0-logloss:20.56262
[0]	validation_0-logloss:0.61913
[182]	val

[I 2023-06-15 15:27:53,088] Trial 1 finished with value: 4.608972343530562 and parameters: {'booster': 'gblinear', 'alpha': 0.047803898808096594, 'lambda': 0.000669631281406815, 'subsample': 0.6855484871090611, 'colsample_bytree': 0.490262726529753}. Best is trial 0 with value: 0.286093718239291.


...

Repeat #1
[0]	validation_0-logloss:1.36002
[101]	validation_0-logloss:17.13552
[0]	validation_0-logloss:0.76786
[120]	validation_0-logloss:20.56262
[0]	validation_0-logloss:1.04871
[109]	validation_0-logloss:0.84322
[0]	validation_0-logloss:0.72684
[250]	validation_0-logloss:0.43913
[0]	validation_0-logloss:1.17248
[104]	validation_0-logloss:17.13552
Repeat #2
[0]	validation_0-logloss:1.15798
[141]	validation_0-logloss:17.58338
[0]	validation_0-logloss:2.85480
[100]	validation_0-logloss:21.41940
[0]	validation_0-logloss:2.16160
[101]	validation_0-logloss:19.70584
[0]	validation_0-logloss:0.77967
[100]	validation_0-logloss:17.13552
[0]	validation_0-logloss:0.85593
[101]	validation_0-logloss:16.27874
Repeat #3
[0]	validation_0-logloss:0.91164
[104]	validation_0-logloss:17.99229
[0]	validation_0-logloss:0.63189
[115]	validation_0-logloss:20.56262
[0]	validation_0-logloss:0.90536
[101]	validation_0-logloss:15.42197
[0]	validation_0-logloss:1.02583
[109]	validation_0-logloss:17.99229
[0]	va

[I 2023-06-15 15:30:51,862] Trial 18 finished with value: 13.230997508388972 and parameters: {'booster': 'gblinear', 'alpha': 0.0004917576090950916, 'lambda': 2.0604867195720236e-06, 'subsample': 0.47268447930073065, 'colsample_bytree': 0.7193344273533461}. Best is trial 14 with value: 0.19042787401206218.


Repeat #1
[0]	validation_0-logloss:0.66288
[1000]	validation_0-logloss:0.21399
[1054]	validation_0-logloss:0.21448
[0]	validation_0-logloss:0.67667
[1000]	validation_0-logloss:0.21456
[1524]	validation_0-logloss:0.21092
[0]	validation_0-logloss:0.67068
[429]	validation_0-logloss:0.37235
[0]	validation_0-logloss:0.66989
[404]	validation_0-logloss:0.31835
[0]	validation_0-logloss:0.67279
[263]	validation_0-logloss:0.35235
Repeat #2
[0]	validation_0-logloss:0.66221
[1000]	validation_0-logloss:0.08273
[1355]	validation_0-logloss:0.08143
[0]	validation_0-logloss:0.68102
[353]	validation_0-logloss:0.16782
[0]	validation_0-logloss:0.68043
[1000]	validation_0-logloss:0.19972
[1065]	validation_0-logloss:0.19994
[0]	validation_0-logloss:0.67902
[166]	validation_0-logloss:0.40022
[0]	validation_0-logloss:0.68016
[1000]	validation_0-logloss:0.18707
[1107]	validation_0-logloss:0.18721
Repeat #3
[0]	validation_0-logloss:0.67903
[942]	validation_0-logloss:0.28831
[0]	validation_0-logloss:0.68143
[100

[W 2023-06-15 15:31:00,963] Trial 19 failed with parameters: {'booster': 'gbtree', 'alpha': 0.9565953476127818, 'lambda': 0.00012312636091171112, 'subsample': 0.9312375839707614, 'colsample_bytree': 0.8585146363436257, 'learning_rate': 0.0429556507861634, 'max_depth': 4, 'min_child_weight': 6, 'gamma': 1.5425985432141796e-07, 'grow_policy': 'depthwise'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/alex/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_5861/4120141353.py", line 75, in objective
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1000)
  File "/home/alex/.local/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/home/alex/.local/lib/python3.10/site-packages/xgboost/sklearn.py", line 1490, in fit
    self._Booster = train(
  File "/home/alex/.local/lib/pytho

KeyboardInterrupt: 

# Load XGBoost parameters

In [21]:
import glob

param_list = glob.glob("optuna_xgb.csv")
models = list()
best_xb_params = list()

xb_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if xb_params.shape[0] == 0:
        xb_params = tmp
    else:
        xb_params = pd.concat([cb_params, tmp])
        
xb_params = xb_params.sort_values('value').head(10)
param_cols = [c for c in xb_params.columns if c.startswith('params_')]
xb_params = xb_params[param_cols]

for idx, row in xb_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['n_estimators'] = 3000
    row_dict['random_state'] = 14062023
    row_dict['early_stopping_rounds'] = 100
    row_dict['verbosity'] = 0
    row_dict['scale_pos_weight'] = 4.71
    row_dict['objective'] = "binary:logistic"
    row_dict['eval_metric'] = "logloss"
    row_dict['tree_method'] = "exact"
    row_dict['booster'] = "gbtree"

    if row_dict["booster"] in ["gbtree", "dart"]:
        row_dict["learning_rate"] = float(row_dict['learning_rate'])
        row_dict["max_depth"] = int(row_dict['max_depth'])
        row_dict["min_child_weight"] = float(row_dict['min_child_weight'])
        row_dict["gamma"] = float(row_dict['gamma'])
    else:
        row_dict["learning_rate"] = None
        row_dict["max_depth"] = None
        row_dict["min_child_weight"] = None
        row_dict["gamma"] = None
        row_dict["grow_policy"] = None     

    if row_dict["booster"] == "dart":
        row_dict["rate_drop"] = float(row_dict['rate_drop'])
        row_dict["skip_drop"] = float(row_dict['skip_drop'])
    else:
        row_dict["sample_type"] = None
        row_dict["normalize_type"] = None
        row_dict["rate_drop"] = None
        row_dict["skip_drop"] = None

    best_xb_params.append(row_dict)

# XGBoost train

In [22]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def xgboost_training():
    models_ = list()
    bll_list = list()
    weights_ = list()
    
    X, y = train_df[features], train_df.Class
#     X, y = generated_features_train, train_df.Class
     
    kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=8062023+20)
    metric = balanced_log_loss
    eval_results_ = {}     # used to store evaluation results for each fold

    oof_level2 = np.zeros([y.shape[0], len(best_xb_params) + 1])
    oof_level2[:, len(best_xb_params)] = y

    print(f"Training with {blu}{X.shape[1]}{res} features")

    for fold, (fit_idx, val_idx) in tqdm(enumerate(kf.split(X=train_df, y=greeks.iloc[:,1:3]), start = 1),
                                         total=CFG.n_stacking_folds):
        
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]

        for i, params in enumerate(best_xb_params):
            
            clf = xgb.XGBClassifier(**params)
            
            clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=500)
            models_.append(clf)

            val_preds = clf.predict_proba(X_val)[:,1]
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = 0

#             print(clf.best_iteration_)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')
        
    return oof_level2, models_

if CFG.stacking:
    oof_level2_xgb, models_xgb = xgboost_training()

Training with 39 features


  0%|          | 0/10 [00:00<?, ?it/s]

[0]	validation_0-logloss:0.65601
[500]	validation_0-logloss:0.05221
[531]	validation_0-logloss:0.05197
Fold:   1| bll_metric: 0.07028 | Best iteration:    0
[0]	validation_0-logloss:0.66491
[500]	validation_0-logloss:0.07772
[735]	validation_0-logloss:0.07377
Fold:   1| bll_metric: 0.09962 | Best iteration:    0
[0]	validation_0-logloss:0.64909
[500]	validation_0-logloss:0.08023
[510]	validation_0-logloss:0.08030
Fold:   1| bll_metric: 0.10457 | Best iteration:    0
[0]	validation_0-logloss:0.66341
[500]	validation_0-logloss:0.07050
[649]	validation_0-logloss:0.07042
Fold:   1| bll_metric: 0.09266 | Best iteration:    0
[0]	validation_0-logloss:0.65339
[500]	validation_0-logloss:0.07362
[508]	validation_0-logloss:0.07433
Fold:   1| bll_metric: 0.08669 | Best iteration:    0
[0]	validation_0-logloss:0.65879
[500]	validation_0-logloss:0.06382
[722]	validation_0-logloss:0.06128
Fold:   1| bll_metric: 0.08373 | Best iteration:    0
[0]	validation_0-logloss:0.66152
[500]	validation_0-loglos

[646]	validation_0-logloss:0.08729
Fold:   5| bll_metric: 0.10013 | Best iteration:    0
[0]	validation_0-logloss:0.66566
[500]	validation_0-logloss:0.10372
[720]	validation_0-logloss:0.10107
Fold:   5| bll_metric: 0.10159 | Best iteration:    0
[0]	validation_0-logloss:0.66404
[500]	validation_0-logloss:0.08916
[656]	validation_0-logloss:0.08924
Fold:   5| bll_metric: 0.09322 | Best iteration:    0
[0]	validation_0-logloss:0.66765
[500]	validation_0-logloss:0.08458
[655]	validation_0-logloss:0.08460
Fold:   5| bll_metric: 0.09224 | Best iteration:    0
[0]	validation_0-logloss:0.66462
[500]	validation_0-logloss:0.09470
[937]	validation_0-logloss:0.08926
Fold:   5| bll_metric: 0.10344 | Best iteration:    0
[0]	validation_0-logloss:0.65429
[385]	validation_0-logloss:0.13522
Fold:   6| bll_metric: 0.23408 | Best iteration:    0
[0]	validation_0-logloss:0.66592
[340]	validation_0-logloss:0.13182
Fold:   6| bll_metric: 0.20671 | Best iteration:    0
[0]	validation_0-logloss:0.65871
[268]	

[232]	validation_0-logloss:0.19921
Fold:  10| bll_metric: 0.57993 | Best iteration:    0
[0]	validation_0-logloss:0.65842
[262]	validation_0-logloss:0.18848
Fold:  10| bll_metric: 0.59030 | Best iteration:    0
[0]	validation_0-logloss:0.66117
[368]	validation_0-logloss:0.17645
Fold:  10| bll_metric: 0.58525 | Best iteration:    0
[0]	validation_0-logloss:0.66031
[261]	validation_0-logloss:0.19464
Fold:  10| bll_metric: 0.61616 | Best iteration:    0
[0]	validation_0-logloss:0.66443
[375]	validation_0-logloss:0.20335
Fold:  10| bll_metric: 0.67076 | Best iteration:    0
[0]	validation_0-logloss:0.66100
[260]	validation_0-logloss:0.19775
Fold:  10| bll_metric: 0.61978 | Best iteration:    0


# CatBoost Optuna optimization

In [67]:
from optuna.integration import CatBoostPruningCallback

X, y = train_df[features], train_df.Class

def objective(trial):
    
    bll_list = list()

    # Parameters
    params = {
        'task_type': 'CPU', # GPU
        'auto_class_weights': 'Balanced',
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
        'eval_metric': 'Logloss',
        'loss_function': 'Logloss', 
        'random_seed': 10062023,
        'od_type': 'Iter', # Type of overfitting detector - stop after k iteraions
        'od_wait': 100, # Overfitting detector - stop training after k iterations without metric improvement
#             'metric_period': 100, # Show metric each k iterations
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
         # Hyperparamters (in order of importance decreasing)
        'iterations' : 5000, # trial.suggest_int('iterations', 300, 1200),        
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 3e-1),    
        'l2_leaf_reg': trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
        'depth' : trial.suggest_int('depth', 4, 10),  # Max tree depth         
         # decrease to deal with overfit
        'subsample': trial.suggest_float('subsample', 0.3, 1), # randomly select part of data without return
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1), # the percentage of features to use at each 
                                                                               # split selection
                                                                               # alias: rsm
                                                                               # not supported in GPU mode
         # decrease to deal with overfit
         'max_leaves': trial.suggest_int('max_leaves', 4, 128), # Max number of leaves in one tree                                                 
         # increase to deal with overfit
        'random_strength': trial.suggest_float('random_strength', 0, 100), # The amount of randomness to use 
                                                                           # for scoring splits when the tree structure
                                                                           # is selected. Helps to avoid overfitting
        'bagging_temperature' : trial.suggest_float('bagging_temperature', 0, 100),     # Assigns random 
                                                                                        # weights to objects
                                                                                        # works only with Bayesian bootstrap
        # this feature value can be increased to 1024 for important features:
        # per_float_feature_quantization='0:border_count=1024'
        'border_count': 254, # trial.suggest_categorical('border_count', [128, 254]), # The number of splits for numerical features
                                                                                      # bigger is better but slowly
                                                                                      # alias: max_bin
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100), # Minimal number of data in one leaf
                                                                           # aliases: min_child_samples, 

    }

    if params['grow_policy'] != 'SymmetricTree': 
        params['boosting_type'] = 'Plain'

    if params['grow_policy'] != 'Lossguide': 
        params['max_leaves'] = None
        
    if params['bootstrap_type'] != 'Bayesian':
        params['bagging_temperature'] = None
    else:
        params['subsample'] = None

#         if params['bootstrap_type'] == 'Bayesian':
#             params['subsample'] = None
#         else:
#             params['subsample'] = trial.suggest_float('subsample', 0.3, 1)

    for i in range(CFG.n_optimize_repeats):
        print(f'Repeat {blu}#{i+1}')

        # Make random under-sampling to balance classes
        positive_count_train = train_df.Class.value_counts()[1]
        sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 
                                                        1: positive_count_train}, 
                                     random_state=15062023+i, 
                                     replacement=True)

        X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']
        
        if CFG.undersample:
            X_re, y_re = sampler.fit_resample(X_re, y_re)
        
        # Create Stratified Multilabel k-Fold scheme
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_feature_sel_folds, shuffle=True, random_state=10062023+i)

        # Create an oof array for inner loop
        oof = np.zeros(X_re.shape[0])

        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X=X_re[features], y=X_re.iloc[:,-3:]), start=1): 
            X, y = X_re[features], y_re
            
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            train_pool = Pool(X_train, y_train, cat_features=['EJ'])
            val_pool = Pool(X_val, y_val, cat_features=['EJ'])

            # Learning
            model = cat.CatBoostClassifier(**params)     
            # Add a callback for pruning
#             pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "Logloss")
            model.fit(train_pool, eval_set=val_pool, verbose=0)#, callbacks=[pruning_callback])
            # Evoke pruning manually
#                 pruning_callback.check_pruned()
            # Predict
            val_preds = model.predict_proba(val_pool)[:,1]
            oof[val_idx] = val_preds
        
        bll_list.append(balanced_log_loss(y_re, oof))    
    
    return np.mean(bll_list)

if CFG.cb_optimize:
#     study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=100), direction="minimize")
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=CFG.n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe()
    df.sort_values('value').iloc[:, [1] + list(range(5, 14))]
    df.to_csv(f'optuna_catboost_fold_.csv')

[I 2023-06-15 15:47:25,150] A new study created in memory with name: no-name-f3c5fd83-32bb-46e0-92c1-970d9dcf1e13


Repeat #1
Repeat #2
Repeat #3
Repeat #4
Repeat #5


[I 2023-06-15 15:48:08,476] Trial 0 finished with value: 0.17049209249813652 and parameters: {'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise', 'learning_rate': 0.006043094362183259, 'l2_leaf_reg': 5.2665170889586595e-08, 'depth': 4, 'subsample': 0.4264901925342959, 'colsample_bylevel': 0.759570287866387, 'max_leaves': 87, 'random_strength': 34.15720934838517, 'bagging_temperature': 66.23907022233848, 'min_data_in_leaf': 34}. Best is trial 0 with value: 0.17049209249813652.


Repeat #1
Repeat #2
Repeat #3


[W 2023-06-15 15:48:26,409] Trial 1 failed with parameters: {'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise', 'learning_rate': 0.008459153699234497, 'l2_leaf_reg': 3.275667642123539e-05, 'depth': 6, 'subsample': 0.8027356041258045, 'colsample_bylevel': 0.4943883271450967, 'max_leaves': 97, 'random_strength': 84.78125060785999, 'bagging_temperature': 13.006780327558431, 'min_data_in_leaf': 46} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home/alex/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_5861/708397832.py", line 104, in objective
    model.fit(train_pool, eval_set=val_pool, verbose=0)#, callbacks=[pruning_callback])
  File "/home/alex/.local/lib/python3.10/site-packages/catboost/core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, N

KeyboardInterrupt: 

# Load CatBoost parameters

In [24]:
import glob

param_list = glob.glob("optuna_catboost*.csv")
models = list()
best_cb_params = list()

cb_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if cb_params.shape[0] == 0:
        cb_params = tmp
    else:
        cb_params = pd.concat([cb_params, tmp])
        
cb_params = cb_params.sort_values('value').head(10)
param_cols = [c for c in cb_params.columns if c.startswith('params_')]
cb_params = cb_params[param_cols]


for idx, row in cb_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['task_type'] = 'CPU'
    row_dict['auto_class_weights'] = 'Balanced'
    row_dict['eval_metric'] = 'Logloss'
    row_dict['loss_function'] = 'Logloss'
    row_dict['random_seed'] = 13062023
    row_dict['verbose'] = 0
    row_dict['od_type'] = 'Iter'
    row_dict['od_wait'] = 100
    row_dict['border_count'] = 254
    row_dict['iterations'] = 10000
    row_dict['bagging_temperature'] = float(row_dict['bagging_temperature'])
    row_dict['subsample'] = 0.7 # float(row_dict['subsample'])
    row_dict['learning_rate'] = float(row_dict['learning_rate'])
    row_dict['l2_leaf_reg'] = float(row_dict['l2_leaf_reg'])
    row_dict['depth'] = int(row_dict['depth'])
    row_dict['random_strength'] = float(row_dict['random_strength'])
    row_dict['min_data_in_leaf'] = int(row_dict['min_data_in_leaf'])
    
    if row_dict['grow_policy'] == 'Lossguide':
        row_dict['max_leaves'] = int(row_dict['max_leaves'])
    else:
        del row_dict['max_leaves']
    
    if row_dict['grow_policy'] == 'SymmetricTree':
        row_dict['boosting_type'] = 'Plain'
    else:
        del row_dict['boosting_type']
        
    if row_dict['task_type'] == 'CPU':
        row_dict['colsample_bylevel'] = 0.6 # float(row_dict['colsample_bylevel'])
    else:
        del row_dict['colsample_bylevel']
    
    best_cb_params.append(row_dict)

# CatBoost train

In [25]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def cb_training():
    models_ = list()
    bll_list = list()
    weights_ = list()
    
    X, y = train_df[features], train_df.Class
#     X, y = generated_features_train, train_df.Class
     
    kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=8062023+20)
    metric = balanced_log_loss
    eval_results_ = {}     # used to store evaluation results for each fold

    oof_level2 = np.zeros([y.shape[0], len(best_cb_params) + 1])
    oof_level2[:, len(best_cb_params)] = y

    print(f"Training with {blu}{X.shape[1]}{res} features")

    for fold, (fit_idx, val_idx) in tqdm(enumerate(kf.split(X=train_df, y=greeks.iloc[:,1:3]), start = 1),
                                         total=CFG.n_stacking_folds):
        
        # Split the dataset according to the fold indexes.
        X_train = X.iloc[fit_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[fit_idx]
        y_val = y.iloc[val_idx]

        train_pool = Pool(X_train, y_train, cat_features=['EJ'])
        val_pool = Pool(X_val, y_val, cat_features=['EJ'])
        
        for i, params in enumerate(best_cb_params):
            
            model = cat.CatBoostClassifier(**params)
            model.fit(train_pool, eval_set=val_pool, verbose=0)
            models_.append(model)

            val_preds = model.predict_proba(val_pool)[:,1]
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = model.best_iteration_

            print(model.best_iteration_)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')
        
    return oof_level2, models_

if CFG.stacking:
    oof_level2_cb, models_cb = cb_training()

Training with 39 features


  0%|          | 0/10 [00:00<?, ?it/s]

6551
Fold:   1| bll_metric: 0.07625 | Best iteration: 6551
9992
Fold:   1| bll_metric: 0.07660 | Best iteration: 9992
6238
Fold:   1| bll_metric: 0.10452 | Best iteration: 6238
3594
Fold:   1| bll_metric: 0.09652 | Best iteration: 3594
9994
Fold:   1| bll_metric: 0.07382 | Best iteration: 9994
6527
Fold:   1| bll_metric: 0.09270 | Best iteration: 6527
2277
Fold:   1| bll_metric: 0.09004 | Best iteration: 2277
1763
Fold:   1| bll_metric: 0.09397 | Best iteration: 1763
5158
Fold:   1| bll_metric: 0.09341 | Best iteration: 5158
5944
Fold:   1| bll_metric: 0.08718 | Best iteration: 5944
4727
Fold:   2| bll_metric: 0.11037 | Best iteration: 4727
5778
Fold:   2| bll_metric: 0.11029 | Best iteration: 5778
3875
Fold:   2| bll_metric: 0.11027 | Best iteration: 3875
4904
Fold:   2| bll_metric: 0.10905 | Best iteration: 4904
3991
Fold:   2| bll_metric: 0.11408 | Best iteration: 3991
5763
Fold:   2| bll_metric: 0.10738 | Best iteration: 5763
3194
Fold:   2| bll_metric: 0.08938 | Best iteration: 31

1151
Fold:   9| bll_metric: 0.23804 | Best iteration: 1151
1645
Fold:   9| bll_metric: 0.24161 | Best iteration: 1645
787
Fold:   9| bll_metric: 0.22050 | Best iteration:  787
827
Fold:   9| bll_metric: 0.19958 | Best iteration:  827
1544
Fold:   9| bll_metric: 0.23594 | Best iteration: 1544
745
Fold:   9| bll_metric: 0.23193 | Best iteration:  745
762
Fold:  10| bll_metric: 0.47249 | Best iteration:  762
905
Fold:  10| bll_metric: 0.45249 | Best iteration:  905
919
Fold:  10| bll_metric: 0.46607 | Best iteration:  919
782
Fold:  10| bll_metric: 0.45930 | Best iteration:  782
870
Fold:  10| bll_metric: 0.45960 | Best iteration:  870
1338
Fold:  10| bll_metric: 0.44404 | Best iteration: 1338
545
Fold:  10| bll_metric: 0.46076 | Best iteration:  545
576
Fold:  10| bll_metric: 0.45785 | Best iteration:  576
1120
Fold:  10| bll_metric: 0.45559 | Best iteration: 1120
578
Fold:  10| bll_metric: 0.46277 | Best iteration:  578


# Stacking with Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

oof_level2 = np.concatenate([oof_level2_lgbm[:,:-1] , oof_level2_cb[:,:-1], oof_level2_xgb[:,:-1]], axis=1)
X = oof_level2
y = oof_level2_lgbm[:,-1]

# mean bll
print(balanced_log_loss(y, np.mean(X, axis=1)))

lr = LogisticRegression(class_weight='balanced')
lr.fit(X, y)

pred = lr.predict_proba(X)[:,1]

# lr bll
print(balanced_log_loss(y, pred))

weights = lr.coef_[0]

0.18570262853231068
0.15867122888336638


In [38]:
len(models_cb)

100

In [27]:
# 0.16616213477809366
# 0.162813150781854

# Models evaluation

In [28]:
# ## Model Evaluation
# metric_score_folds = pd.DataFrame.from_dict(all_eval_results_)
# fit_logloss = []
# val_logloss = []

# for seed in CFG.seeds:
#     for fold in range(1,CFG.n_folds+1):
#         fit_logloss.append(metric_score_folds[seed][fold]['training']['balanced_log_loss'])
#         val_logloss.append(metric_score_folds[seed][fold]['valid_1']['balanced_log_loss'])

# fig, axes = plt.subplots(math.ceil(CFG.n_folds*len(CFG.seeds)/CFG.n_folds), CFG.n_folds, figsize=(20, 20), dpi=150)
# ax = axes.flatten()
# for i, (f, v, m) in enumerate(zip(fit_logloss, val_logloss, models_), start = 1): 
#     sns.lineplot(f, color='#B90000', ax=ax[i-1], label='fit')
#     sns.lineplot(v, color='#048BA8', ax=ax[i-1], label='val')
#     ax[i-1].legend()
#     ax[i-1].spines['top'].set_visible(False);
#     ax[i-1].spines['right'].set_visible(False)
#     ax[i-1].set_title(f'Seed {CFG.seeds[(i-1)//CFG.n_folds]} Fold {CFG.n_folds if i%CFG.n_folds==0 else i%CFG.n_folds}', fontdict={'fontweight': 'bold'})

#     color =  ['#048BA8', palette[-3]]
#     best_iter = m.best_iteration_
#     span_range = [[0, best_iter], [best_iter + 10, best_iter + CFG.num_boost_round]]

#     for idx, sub_title in enumerate([f'Best\nIteration: {best_iter}', f'Early\n Stopping: 2000']):
#         ax[i-1].annotate(sub_title,
#                     xy=(sum(span_range[idx])/2 , 0.5),
#                     xytext=(0,0), textcoords='offset points',
#                     va="center", ha="center",
#                     color="w", fontsize=16, fontweight='bold',
#                     bbox=dict(boxstyle='round4', pad=0.4, color=color[idx], alpha=0.6))
#         ax[i-1].axvspan(span_range[idx][0]-0.4,span_range[idx][1]+0.4,  color=color[idx], alpha=0.07)

#     ax[i-1].set_xlim(0, best_iter + 20 + 2000)
#     ax[i-1].legend(bbox_to_anchor=(0.95, 1), loc='upper right', title='logloss')

# plt.tight_layout();

# Predict test

In [29]:
models = models_lgbm # + models_cb

def predict(X):
    y = np.zeros(len(X))
    for i, model in enumerate(models):
#         y += weights[i] * model.predict_proba(X)[:,1]
        y += model.predict_proba(X)[:,1]
#     return y / sum(weights)
    return y / len(models)

predictions = predict(test_df[features])
# predictions = predict(generated_features_test)

test_df['class_1'] = predictions
test_df['class_0'] = 1 - predictions

sample_submission[['class_0', 'class_1']] = test_df[['class_0', 'class_1']]
sample_submission.to_csv(r"submission.csv", index=False)
sample_submission

,Id,class_0,class_1
0,00eed32682bb,0.696636,0.303364
1,010ebe33f668,0.696636,0.303364
2,02fa521e1838,0.696636,0.303364
3,040e15f562a2,0.696636,0.303364
4,046e85c7cc7f,0.696636,0.303364


You have a lot of resulting features. I have already identified a few important once. 